In [1]:
# 조회? 하는 최근접 취약시설의 수
n_neighbors = 5

In [2]:
# !pip install --break-system-packages scikit-learn
# !pip install --break-system-packages math

# !pip install scikit-learn
# !pip install math
# 필요한 라이브러리 임포트

In [3]:
import numpy as np
from sklearn.neighbors import KDTree
from math import radians, sin, cos, sqrt, atan2
import pandas as pd

In [4]:
# 데이터 불러오기
road_url = '../광진구 열선 위치 분석/make_file/(최종)열선_위경도값(중앙값_고도_경사각_포함).csv'
facility_url = '../광진구 취약시설 위치분석/make_file/광진구_모든_시설의_위치.csv'

In [5]:
facility_data = pd.read_csv(facility_url)
road_data = pd.read_csv(road_url, encoding = "EUC_KR")

In [6]:
facility_data.columns

Index(['Unnamed: 0', '시설_구분', '시설_이름', '시설_위치', '위도', '경도'], dtype='object')

In [7]:
road_data.columns

Index(['Unnamed: 0.1', '행정동', '노선명', '기점', '종점', '기점_좌표_위도', '기점_좌표_경도',
       '종점_좌표_위도', '종점_좌표_경도', '열선_중앙_위도', '열선_중앙_경도', '시작점_고도', '종료점_고도',
       '고도의_차이', '경사각', 'Unnamed: 0', '열선연장(m) (1차로 기준)', '차로수'],
      dtype='object')

In [8]:

# 도로 중앙값 데이터 (위경도)
road_lat_list = road_data['종점_좌표_위도'].to_list()
road_lon_list = road_data['종점_좌표_경도'].to_list()

# 취약시설 데이터 (시설명, 위경도) 
facility_lat_list = facility_data["위도"].to_list()
facility_lon_list = facility_data["경도"].to_list()


In [9]:
print(f"road_lat_list 길이 : {len(road_lat_list)}")
print(f"road_lon_list 길이 : {len(road_lon_list)}")
 
print(f"facility_lat_list 길이 : {len(facility_lat_list)}")
print(f"facility_lon_list 길이 : {len(facility_lon_list)}")

road_lat_list 길이 : 41
road_lon_list 길이 : 41
facility_lat_list 길이 : 190
facility_lon_list 길이 : 190


In [10]:
# 거리를 구하기
def haversine(lat1, lon1, lat2, lon2): 
    R = 6371  # 지구 반경 (km)
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    result = R * c * 1000 
    return result # 미터 단위로 반환

In [11]:
def find_nearest_facilities_with_haversine(road_lats, road_lons, facility_lats, facility_lons):
    global n_neighbors
    # KDTree로 시설 좌표 인덱스를 트리 구조로 저장
    facilities = np.array(list(zip(facility_lats, facility_lons)))
    tree = KDTree(facilities)

    # 각 도로에 대해 최근접 n개의 시설 찾기
    distance_matrix = []
    avg_distances = []
    
    for r_lat, r_lon in zip(road_lats, road_lons):
        road_coords = np.array([[r_lat, r_lon]])
        
        # 최근접 n개 시설의 인덱스 및 거리 계산
        dist, indices = tree.query(road_coords, k=n_neighbors)
        
        # 거리 계산하여 2차원 배열에 저장
        distances = []
        for i in range(n_neighbors):
            dist_val = haversine(r_lat, r_lon, facility_lats[indices[0][i]], facility_lons[indices[0][i]])  # Haversine 거리 계산
            distances.append(dist_val)
        
        distance_matrix.append(distances)
        
        # 평균 거리 계산하여 저장
        avg_distance = np.mean(distances)
        avg_distances.append(avg_distance)
    
    return np.array(distance_matrix), np.array(avg_distances)

In [12]:
# 거리 계산
distance_matrix, avg_distances = find_nearest_facilities_with_haversine(road_lat_list, road_lon_list, facility_lat_list, facility_lon_list)

In [13]:
print(f"수집한 도로_취약시설 거리의 수 : {len(distance_matrix)}")
print(f"수집한 도로_취약시설 거리의 평균 수 : {len(avg_distances)}")

수집한 도로_취약시설 거리의 수 : 41
수집한 도로_취약시설 거리의 평균 수 : 41


In [14]:
print(distance_matrix[1])

[ 77.3951541  117.20156943 194.4967099  235.8183382  296.9583392 ]


In [15]:
distance_matrix = distance_matrix.tolist()

In [16]:
new_distance_matrix = [" ,".join(map(str, row)) for row in distance_matrix]
new_distance_matrix[1]

'77.39515409637183 ,117.20156943020137 ,194.49670989697435 ,235.81833820454924 ,296.9583391993342'

In [17]:
road_data['최근접_시설들_거리'] = new_distance_matrix
road_data['최근접_시설의_평균거리'] = avg_distances

In [18]:
print(road_data['최근접_시설의_평균거리'].mean())

276.3241670104214


In [19]:
# road_data['최근접_시설들_거리'] 

In [20]:
road_data.columns


Index(['Unnamed: 0.1', '행정동', '노선명', '기점', '종점', '기점_좌표_위도', '기점_좌표_경도',
       '종점_좌표_위도', '종점_좌표_경도', '열선_중앙_위도', '열선_중앙_경도', '시작점_고도', '종료점_고도',
       '고도의_차이', '경사각', 'Unnamed: 0', '열선연장(m) (1차로 기준)', '차로수', '최근접_시설들_거리',
       '최근접_시설의_평균거리'],
      dtype='object')

In [21]:
save_file_url = './make_file/열선_취약시설_거리.csv'
save_file_encoding = 'EUC-KR'
# save_file_encoding = 'UTF-8'

In [22]:
try :
    road_data.to_csv(save_file_url, encoding=save_file_encoding)
    print(f"{save_file_url} 파일 저장을 성공했습니다.")
except OSError as e:
    print(e)

./make_file/열선_취약시설_거리.csv 파일 저장을 성공했습니다.
